#MODELO: RANDOM FOREST


## 1. CONFIGURACIÓN E IMPORTACIONES

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc
import warnings

# Limpiar salidas molestas
warnings.filterwarnings('ignore')



## 2. CARGA DE DATOS


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

!unzip udea*.zip > /dev/null
!wc *.csv

print("Cargando datasets para Random Forest...")
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.06GB/s]
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total
Cargando datasets para Random Forest...



## 3. PREPROCESAMIENTO

In [4]:
def limpieza_random_forest(df_input):
    df = df_input.copy()

    # A. Eliminar columnas con un solo valor (no aportan info)
    for col in df.columns:
        if df[col].nunique(dropna=False) <= 1:
            df.drop(columns=[col], inplace=True)

    # B. Ingeniería de Características Básica
    # Suma de educación de padres (Capital Educativo)
    # Mapeamos primero para poder sumar
    mapa_edu = {
        'Ninguno': 0, 'No sabe': 0, 'Primaria incompleta': 1, 'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
        'Postgrado': 7
    }
    col_padres = ['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']
    if all(c in df.columns for c in col_padres):
        # Llenamos temporalmente con 0 para sumar
        edu_padre = df['F_EDUCACIONPADRE'].map(mapa_edu).fillna(0)
        edu_madre = df['F_EDUCACIONMADRE'].map(mapa_edu).fillna(0)
        df['CAPITAL_EDUCATIVO_RF'] = edu_padre + edu_madre

    # C. Tratamiento de Nulos y Categóricas
    # Iteramos sobre todas las columnas
    for col in df.columns:
        # 1. Si es numérica (float/int)
        if pd.api.types.is_numeric_dtype(df[col]):
            # Rellenar nulos con un valor fuera de rango (-999)
            # Esto ayuda al árbol a separar los que tenían datos de los que no
            df[col] = df[col].fillna(-999)

        # 2. Si es objeto (texto) o categoría
        else:
            # Rellenar nulos con string explícito
            df[col] = df[col].fillna('SIN_DATO')
            # Convertir a números (Label Encoding rápido)
            df[col] = df[col].astype('str').astype('category').cat.codes

    return df

print("Procesando datos (rellenando nulos y codificando)...")

# Separamos X e y
X_raw = df_train.drop(columns=['RENDIMIENTO_GLOBAL', 'ID'])
X_submission_raw = df_test.drop(columns=['ID'])
y_raw = df_train['RENDIMIENTO_GLOBAL']
ids_test = df_test['ID']

# Aplicamos la limpieza
X = limpieza_random_forest(X_raw)
X_sub = limpieza_random_forest(X_submission_raw)

# Codificar Target
le = LabelEncoder()
y = le.fit_transform(y_raw)

# Limpieza de memoria
del df_train, df_test, X_raw, X_submission_raw
gc.collect()

print(f"Dimensiones listas: {X.shape}")


Procesando datos (rellenando nulos y codificando)...
Dimensiones listas: (692500, 20)



## 4. CONFIGURACIÓN DEL MODELO

In [5]:
# Definimos el Random Forest
# Ajustamos parámetros para que no tarde demasiado pero sea preciso
bosque = RandomForestClassifier(
    n_estimators=300,        # Número de árboles (300 es robusto)
    criterion='gini',        # Criterio de división estándar
    max_depth=18,            # Profundidad máxima (evita archivos gigantes y lentitud)
    min_samples_split=5,     # Mínimo de datos para dividir un nodo
    min_samples_leaf=2,      # Mínimo de datos en una hoja
    max_features='sqrt',     # Número de features a considerar en cada división
    bootstrap=True,
    n_jobs=-1,               # Usar todos los núcleos del CPU
    random_state=123,        # Semilla diferente a los otros modelos
    verbose=0
)

## 5. VALIDACIÓN (Split 80/20)

In [6]:
print("\n--- Fase 1: Validación Interna ---")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

print("Entrenando bosque en set de validación...")
bosque.fit(X_train, y_train)

# Evaluación
preds_val = bosque.predict(X_val)
acc = accuracy_score(y_val, preds_val)
print(f">>> Accuracy Estimado (Random Forest): {acc:.5f}")


--- Fase 1: Validación Interna ---
Entrenando bosque en set de validación...
>>> Accuracy Estimado (Random Forest): 0.41126


## 6. ENTRENAMIENTO FINAL Y SUBMISSION

In [7]:
print("\n--- Fase 2: Entrenamiento con dataset COMPLETO ---")
# Entrenamos con TODO (X, y) para el archivo final
bosque.fit(X, y)

print("Generando predicciones finales...")
preds_codigos = bosque.predict(X_sub)
preds_etiquetas = le.inverse_transform(preds_codigos)

# Guardar Archivo
nombre_archivo = 'submission_random_forest.csv'
df_export = pd.DataFrame({'ID': ids_test, 'RENDIMIENTO_GLOBAL': preds_etiquetas})
df_export.to_csv(nombre_archivo, index=False)

print(f"¡Hecho! Archivo generado: {nombre_archivo}")


--- Fase 2: Entrenamiento con dataset COMPLETO ---
Generando predicciones finales...
¡Hecho! Archivo generado: submission_random_forest.csv
